<a href="https://colab.research.google.com/github/sayed-Gomaa/Neural-Network-form-scratch-/blob/main/practice%20on%20frist%20DL_course.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [5]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.compose import ColumnTransformer
lebel_encode=LabelEncoder()
X[:,2]=lebel_encode.fit_transform(X[:,2])

In [6]:
columnTransformer=ColumnTransformer([('encoder',OneHotEncoder(),[1])],remainder='passthrough')
X = np.array(columnTransformer.fit_transform(X), dtype = np.str)

In [7]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [8]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [71]:
def sigmoid(Z):
     
    A = 1/(1+np.exp(-Z))
    cache = Z
    
    return A, cache

def relu(Z):
    
    
    A = np.maximum(0,Z)
    
    assert(A.shape == Z.shape)
    
    cache = Z 
    return A, cache


def relu_backward(dA, cache):
     
    Z = cache
    dZ = np.array(dA, copy=True) # just converting dz to a correct object.
    
    # When z <= 0, you should set dz to 0 as well. 
    dZ[Z <= 0] = 0
    
    assert (dZ.shape == Z.shape)
    
    return dZ

def sigmoid_backward(dA, cache):
    """
    Implement the backward propagation for a single SIGMOID unit.
    Arguments:
    dA -- post-activation gradient, of any shape
    cache -- 'Z' where we store for computing backward propagation efficiently
    Returns:
    dZ -- Gradient of the cost with respect to Z
    """
    
    Z = cache
    
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)
    
    assert (dZ.shape == Z.shape)
    
    return dZ

In [45]:
def init_params(layer_dims):
  params={}
  L=len(layer_dims)
  for l in range(1,L):
    params['W'+str(l)]=np.random.randn(layer_dims[l],layer_dims[l-1])
    params['b'+str(l)]=np.zeros((layer_dims[l],1))
  return params

In [68]:
def feedfworward(X, paramters):
   
    caches = []
    A = X
    L = len(paramters) // 2                  # number of layers in the neural network
    
  
    for l in range(1, L):

        A_prev = A 
        Z=np.dot(paramters['W'+str(l)],A_prev)+paramters['b'+str(l)]
        A,activation_cache=relu(Z)
        
        linear_cache = (A_prev, paramters['W'+str(l)], paramters['b'+str(l)])
        cache=(linear_cache,activation_cache)
        caches.append(cache)
        
    #output layer
    Z=np.dot(paramters['W'+str(L)],A)+paramters['b'+str(L)]
    AL,activation_cache=sigmoid(Z)
    linear_cache = (A, paramters['W'+str(L)], paramters['b'+str(L)])
    cache=(linear_cache,activation_cache)
    caches.append(cache)
      
    return AL, caches

In [47]:
def compute_cost(AL, Y):
    m = len(Y)
    
    cost= -1/m * np.sum(np.dot(Y,np.log(AL).T) + np.dot((1-Y),np.log(1-AL).T) )
    
    return cost

In [77]:
 def feedbackward(AL, Y, caches):
    
    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
     
    # Initializing the backpropagation
    dAL =- (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL)) 
     
    
    # Lth layer (SIGMOID -> LINEAR) gradients. 
    
    linear_cache, activation_cache = caches[L-1]
    dZ = sigmoid_backward(dAL, activation_cache) 
    A_prev, W, b = linear_cache
     
    m = len(A_prev)
    dW = 1/m * np.dot(dZ, A_prev.T)
    db =1/m * np.sum (dZ, axis=1,keepdims=True)
    dA_prev = np.dot( W.T, dZ)
    grads["dA" + str(L-1)] = dA_prev
    grads["dW" + str(L)] = dW
    grads["db" + str(L)] = db 
    

    # Loop from l=L-2 to l=0
    for l in reversed(range(L-1)):
        # lth layer: (RELU -> LINEAR) gradients.
        linear_cache, activation_cache = caches[l]
        dZ  = relu_backward(grads["dA" + str(l + 1)], activation_cache) 
        
        A_prev, W, b = linear_cache
        m = len(A_prev)
        
        dW = 1/m * np.dot(dZ, A_prev.T)
        db =1/m * np.sum (dZ, axis=1,keepdims=True)
        dA_prev = np.dot( W.T, dZ)
        
        grads["dA" + str(l)] = dA_prev
        grads["dW" + str(l + 1)] = dW
        grads["db" + str(l + 1)] = db
      

        
    return grads

In [49]:
def update_parameters(params, grads, learning_rate):
    parameters = params.copy()
    L = len(parameters) // 2 # number of layers in the neural network
 
    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * grads["dW" + str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate *grads["db" + str(l+1)]
       
    return parameters

In [75]:
def L_layer_model(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost=False):
    costs = []                              
    parameters = init_params(layers_dims)
    
    
    # Loop (gradient descent)
    for i in range(0, num_iterations):
        AL, caches =feedfworward(X, parameters)
         
        # Compute cost.
        cost =  compute_cost(AL, Y)
    
        # Backward propagation.
        grads = feedbackward(AL, Y, caches)
       
        # Update parameters.
        parameters = update_parameters(parameters, grads, learning_rate)
                
        # Print the cost every 100 iterations
        if print_cost and i % 100 == 0 or i == num_iterations - 1:
            print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))
        if i % 100 == 0 or i == num_iterations:
            costs.append(cost)
    
    return parameters, costs

In [80]:
layers_dims = [12, 5, 1] #  3-layer model
parameters, costs = L_layer_model(X_train.T,y_train, layers_dims, num_iterations = 1000, print_cost = True)

Cost after iteration 0: 1.0815913265877184
Cost after iteration 100: 0.4138278476122523
Cost after iteration 200: 0.40274553546648223
Cost after iteration 300: 0.39968918966861355
Cost after iteration 400: 0.3981306203405105
Cost after iteration 500: 0.3965494217587824
Cost after iteration 600: 0.3965938347603027
Cost after iteration 700: 0.3966800714386534
Cost after iteration 800: 0.395271187840922
Cost after iteration 900: 0.39522767435385325
Cost after iteration 999: 0.39962184946753415
